In [1]:
# !ls ../../cloze_answer
# !pip install xlrd -i https://pypi.douban.com/simple

In [1]:
import json
import numpy as np
import pandas as pd
from random import choice
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
import re, os
import codecs


Using TensorFlow backend.


In [2]:
maxlen = 100
config_path = '../../cloze_answer/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '../../cloze_answer/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '../../cloze_answer/chinese_L-12_H-768_A-12/vocab.txt'


token_dict = {}

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

In [4]:
list(token_dict.items())[0:10]

[('', 13503),
 ('秋', 4904),
 ('##映', 16273),
 ('##29', 8887),
 ('苛', 5729),
 ('##穫', 18010),
 ('##麩', 20989),
 ('##卮', 14368),
 ('##玄', 17428),
 ('ᄂ', 290)]

In [83]:
token_dict['''[PAD]''']##原始pad对应0，直接使用预训练模型，pad也要对应0

0

In [84]:
token_dict['''[UNK]''']

100

In [85]:
token_dict['''[unused1]''']

1

In [86]:
for each in ['''[UNK]''','''[CLS]''','''[SEP]''','''[MASK]''']:
    print(each,token_dict[each])

[UNK] 100
[CLS] 101
[SEP] 102
[MASK] 103


In [3]:
class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

In [39]:
Tokenizer(token_dict).encode(first='东西不错，售后太差，安装一个热水器400块钱，像话吗？钱给完了发现被黑了！')

([101,
  691,
  6205,
  679,
  7231,
  8024,
  1545,
  1400,
  1922,
  2345,
  8024,
  2128,
  6163,
  671,
  702,
  4178,
  3717,
  1690,
  8230,
  1779,
  7178,
  8024,
  1008,
  6413,
  1408,
  8043,
  7178,
  5314,
  2130,
  749,
  1355,
  4385,
  6158,
  7946,
  749,
  8013,
  102],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0])

In [38]:
##bert自带的分词方式
Tokenizer(token_dict).tokenize(first='东西龘尕尕，售后 太差，安装一个热水器400块钱，像话吗？钱给完了发现被黑了！')

['[CLS]',
 '东',
 '西',
 '龘',
 '尕',
 '尕',
 '，',
 '售',
 '后',
 '太',
 '差',
 '，',
 '安',
 '装',
 '一',
 '个',
 '热',
 '水',
 '器',
 '400',
 '块',
 '钱',
 '，',
 '像',
 '话',
 '吗',
 '？',
 '钱',
 '给',
 '完',
 '了',
 '发',
 '现',
 '被',
 '黑',
 '了',
 '！',
 '[SEP]']

In [36]:
tokenizer.tokenize(first='东西龘尕尕，售后 太差，安装一个热水器400块钱，像话吗？钱给完了发现被黑了！')
##做了修改对应在训练语料中没有出现的词用[unk]代替，空格用[unused1]代替

['[CLS]',
 '东',
 '西',
 '[UNK]',
 '尕',
 '尕',
 '，',
 '售',
 '后',
 '[unused1]',
 '太',
 '差',
 '，',
 '安',
 '装',
 '一',
 '个',
 '热',
 '水',
 '器',
 '4',
 '0',
 '0',
 '块',
 '钱',
 '，',
 '像',
 '话',
 '吗',
 '？',
 '钱',
 '给',
 '完',
 '了',
 '发',
 '现',
 '被',
 '黑',
 '了',
 '！',
 '[SEP]']

In [17]:
indices, segments = tokenizer.encode(first='东西不错，售后太差，安装一个热水器400块钱，像话吗？钱给完了发现被黑了！')
print('indices={}'.format(indices),'\n','segments={}'.format(segments))

indices=[101, 691, 6205, 679, 7231, 8024, 1545, 1400, 1922, 2345, 8024, 2128, 6163, 671, 702, 4178, 3717, 1690, 125, 121, 121, 1779, 7178, 8024, 1008, 6413, 1408, 8043, 7178, 5314, 2130, 749, 1355, 4385, 6158, 7946, 749, 8013, 102] 
 segments=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
!ls

neg.xls  pos.xls  sent_classifier.ipynb


In [4]:
neg = pd.read_excel('neg.xls', header=None)
pos = pd.read_excel('pos.xls', header=None)






In [63]:
neg.head()

,0
0,做为一本声名在外的流行书，说的还是广州的外企，按道理应该和我的生存环境差不多啊。但是一看之下...
1,作者有明显的自恋倾向，只有有老公养不上班的太太们才能像她那样生活。很多方法都不实用，还有抄袭...
2,作者完全是以一个过来的自认为是成功者的角度去写这个问题，感觉很不客观。虽然不是很喜欢，但是，...
3,作者提倡内调，不信任化妆品，这点赞同。但是所列举的方法太麻烦，配料也不好找。不是太实用。
4,作者的文笔一般，观点也是和市面上的同类书大同小异，不推荐读者购买。


In [65]:
neg[0]

0        做为一本声名在外的流行书，说的还是广州的外企，按道理应该和我的生存环境差不多啊。但是一看之下...
1        作者有明显的自恋倾向，只有有老公养不上班的太太们才能像她那样生活。很多方法都不实用，还有抄袭...
2        作者完全是以一个过来的自认为是成功者的角度去写这个问题，感觉很不客观。虽然不是很喜欢，但是，...
3             作者提倡内调，不信任化妆品，这点赞同。但是所列举的方法太麻烦，配料也不好找。不是太实用。
4                        作者的文笔一般，观点也是和市面上的同类书大同小异，不推荐读者购买。
5        作者的文笔还行，但通篇感觉太琐碎，有点文人的无病呻吟。自由主义者。作者的品性不敢苟同，无民族...
6                            作者倒是个很小资的人,但有点自恋的感觉,书并没有什么大帮助
7        作为一本描写过去年代感情生活的小说，作者明显生活经验不足，并且文字功底极其一般，看后感觉浪费...
8        作为个人经验在网上谈谈可以，但拿来出书就有点过了，书中还有些明显的谬误。不过文笔还不错，建议...
9        昨天刚兴奋地写了评论,今天便遇一闹心事,因把此套书推荐给很多朋友,朋友就拖我在网上购,结果前...
10                              昨天打开书才翻了几页，书页纷纷掉落了，请问怎么回事？
11       最近下单号为：1442083355，其中一套书《易经的智慧》当当网  少带光盘 。为了得到解...
12                                     最大的缺陷是容易诱导养成中文式英语习惯
13                        最大的困扰是穴位有点把不准，不过我牢记中里先生离穴不离经的教诲。
14       最大的不好就是没有音标，看的时候不是很方便，而且书的内容一点也不好笑，或者说作者幽默感很有限。。。
15       纵观整部书（上下两册）  从文字，到结构，人物，情节 没有一个地方是可取的虽然有过从业经验 ...
16                    总结一下，女人要现实。要么嫁好老公，要么抓住机遇有事业……这还用她说

In [64]:
pos.head()

,0
0,做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...


In [5]:
data = []

##没有字段名称就用序号0表示
#负例标签0，正例标签1
for d in neg[0]:
    data.append((d, 0))

for d in pos[0]:
    data.append((d, 1))

In [6]:
np.random.shuffle(data)
#data顺序已经被打乱

In [72]:
data[0:20]

[('不好玩感觉买它是种浪费如果没有认真看它,把它当小说一样看的热情还是不要买了不值得', 0),
 ('性能非常棒，使命召唤4，5，波斯王子4，极品飞车12效果最高，2倍防锯齿都能流畅运行', 1),
 ('有些例子不够典型，有滥竽充数之嫌，希望看到的是一些不这么幼稚和明显的中式英语（一般情况下略略学过英语的人都不会犯书中所举的低级错误），应该写一些类似文化差异所造成的“笑话”，如blue story，大多数人只知道blue是忧伤的意思，其实这里变成了黄色小说的意思了。从排版来说，该书还是有可取的地方，看起来比较轻松，有些还是比较好笑的。',
  0),
 ('无express插槽，无e-sata, 屏幕可视角太小，色域也只能算凑合，转轴处塑料感偏重', 0),
 ('在不敢喝奶，唯恐一不小心被蒙牛，伊利「毒死」的年代，路边阿婆的老酸奶让你放下胃蕾的戒备，难得放心喝一口。', 0),
 ('房间很冷且不隔音，服务一般，卫生间很小，房间整体舒适性在我住过的如家是最差的。', 0),
 ('usb接口的位置不怎么好，一个在后面，两个在左边，一般鼠标都在右边，就只有插在后面那个usb接口上，设计有点小问题', 0),
 ('酒店一般般。 服务态度也不是很好， 主要是人家太忙了。需要等很长时间。要不是要办事里那里很近； 应该不会再住这个了。没有停车的地方。只能停在旁边的有个很大的洗浴的地方。不过那里不错，有兴趣的朋友可以去试试：）',
  0),
 ('介绍说是很好，看了之后，名不副实，说教，空洞。没有实际的事例远远不如另外一本书《遇见未知的自己》好。', 0),
 (' 酒店位于市中心，比较方便，房间设施较一般，最苦恼的是，说是市政府规定，室外温度不过27度就不能开空调，所以白天几乎不开空调，热得我们没处躲，要知道这可是宾馆不是招待所，房间都是不对流的，不开空调能闷死人呢，真难受啊，还好到晚上空调开了，不然我就立马退房走人了。本市的地图是要买的，不像别的酒店都有赠送。',
  0),
 ('送货快算是个优点吧，我是第一次在京东上买东西，感觉像受骗了一样。京东还号称搞活动返券，结果返了150也不能用，提示说只能消费三千才能用这150；像上了贼船一样。',
  1),
 ('最近受几位老年大学学友的委托，又购买了几本《数码单反摄影轻松入门》和《RAW完全解析》，确实很不错！

In [8]:
# 按照9:1的比例划分训练集和验证集
random_order = range(len(data))
random_order

range(0, 21105)

In [9]:

# np.random.shuffle(data)

#划分训练集和验证集
train_data = [data[j] for i, j in enumerate(random_order) if i % 10 != 0]
valid_data = [data[j] for i, j in enumerate(random_order) if i % 10 == 0]

In [75]:
train_data[0:10]

[('性能非常棒，使命召唤4，5，波斯王子4，极品飞车12效果最高，2倍防锯齿都能流畅运行', 1),
 ('有些例子不够典型，有滥竽充数之嫌，希望看到的是一些不这么幼稚和明显的中式英语（一般情况下略略学过英语的人都不会犯书中所举的低级错误），应该写一些类似文化差异所造成的“笑话”，如blue story，大多数人只知道blue是忧伤的意思，其实这里变成了黄色小说的意思了。从排版来说，该书还是有可取的地方，看起来比较轻松，有些还是比较好笑的。',
  0),
 ('无express插槽，无e-sata, 屏幕可视角太小，色域也只能算凑合，转轴处塑料感偏重', 0),
 ('在不敢喝奶，唯恐一不小心被蒙牛，伊利「毒死」的年代，路边阿婆的老酸奶让你放下胃蕾的戒备，难得放心喝一口。', 0),
 ('房间很冷且不隔音，服务一般，卫生间很小，房间整体舒适性在我住过的如家是最差的。', 0),
 ('usb接口的位置不怎么好，一个在后面，两个在左边，一般鼠标都在右边，就只有插在后面那个usb接口上，设计有点小问题', 0),
 ('酒店一般般。 服务态度也不是很好， 主要是人家太忙了。需要等很长时间。要不是要办事里那里很近； 应该不会再住这个了。没有停车的地方。只能停在旁边的有个很大的洗浴的地方。不过那里不错，有兴趣的朋友可以去试试：）',
  0),
 ('介绍说是很好，看了之后，名不副实，说教，空洞。没有实际的事例远远不如另外一本书《遇见未知的自己》好。', 0),
 (' 酒店位于市中心，比较方便，房间设施较一般，最苦恼的是，说是市政府规定，室外温度不过27度就不能开空调，所以白天几乎不开空调，热得我们没处躲，要知道这可是宾馆不是招待所，房间都是不对流的，不开空调能闷死人呢，真难受啊，还好到晚上空调开了，不然我就立马退房走人了。本市的地图是要买的，不像别的酒店都有赠送。',
  0),
 ('最近受几位老年大学学友的委托，又购买了几本《数码单反摄影轻松入门》和《RAW完全解析》，确实很不错！', 1)]

In [10]:

##进行padding补零处理;这里似乎padding之后没有经过mask处理在计算loss的时候；
##原始bert中pad对应0，直接使用预训练模型，pad也要对应0
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])


class data_generator:
    def __init__(self, data, batch_size=32):
        self.data = data
        '''
        data数据形式如下：
        [('性能非常棒，使命召唤4，5，波斯王子4，极品飞车12效果最高，2倍防锯齿都能流畅运行', 1),
         ('有些例子不够典型，有滥竽充数之嫌，希望看到的是一些不这么幼稚和明显的中式英语（一般情况下略略学过英语的人都不会犯书中所举的低级错误），应该写一些类似文化差异所造成的“笑话”，如blue story，大多数人只知道blue是忧伤的意思，其实这里变成了黄色小说的意思了。从排版来说，该书还是有可取的地方，看起来比较轻松，有些还是比较好笑的。',
  0),
        '''
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        #取整后如果不够在多加一步
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = range(len(self.data))#range(0, 21105)
            np.random.shuffle(self.data)#将data顺序打乱
            X1, X2, Y = [], [], []
            #根据序号取出每一个数据
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen]#序号0对应文本内容，且截取最长字符个数；1对应类别标签
                #数据编码；得到句中每个字的id,以及该句话的id(0第一句，1第二句)
                x1, x2 = tokenizer.encode(first=text)
                '''
                x1=indices=[101, 691, 6205, 679, 7231, 8024, 1545, 1400, 1922, 2345, 8024, 2128, 6163, 671, 702, 4178, 3717, 1690, 125, 121, 121, 1779, 7178, 8024, 1008, 6413, 1408, 8043, 7178, 5314, 2130, 749, 1355, 4385, 6158, 7946, 749, 8013, 102] 
                x2=segments=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                '''
                
                y = d[1]#1对应类别标签
                X1.append(x1)#将每句话中所有字的indices组成的列表放到一个新列表里
                X2.append(x2)#将每句话中所有字的segments组成的列表放到一个新列表里
                Y.append([y])#每个标签组成的列表放到新列表里
                #满足一个batch或者结束了就返回
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    ##返回格式如下，X1，X2，Y都是二维列表
                    yield [X1, X2], Y
                    #每一个批次返回后就置空
                    [X1, X2, Y] = [], [], []




In [11]:
train_D = data_generator(train_data)
valid_D = data_generator(valid_data)

In [88]:
import tensorflow as tf
g=train_D.__iter__()#batch_size=32,返回一个batch的数据
d=next(g)

In [89]:

d

([array([[ 101,  671, 4684, ..., 5491, 5489,  102],
         [ 101, 6821, 3315, ..., 2190, 3124,  102],
         [ 101, 3341, 5722, ..., 2697, 6230,  102],
         ...,
         [ 101,  671,  702, ...,    0,    0,    0],
         [ 101,  779, 4263, ..., 6632, 3341,  102],
         [ 101, 5709,  749, ..., 1431, 8024,  102]]),
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]])],
 array([[1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1]]))

In [78]:
d[0][0].shape

TensorShape([32, 102])

In [79]:
d[0][1].shape

TensorShape([32, 102])

In [80]:
d[1].shape

TensorShape([32, 1])

In [81]:
[x1_in, x2_in]=d


In [84]:
x1_in

[<tf.Tensor: id=90799, shape=(32, 102), dtype=int64, numpy=
 array([[ 101,    1, 5885, ...,    0,    0,    0],
        [ 101,  147,  124, ...,    0,    0,    0],
        [ 101,  100,  100, ...,    0,    0,    0],
        ...,
        [ 101, 4696, 3221, ..., 1990, 2823,  102],
        [ 101, 6968, 6228, ...,    0,    0,    0],
        [ 101,  966, 2533, ...,  741, 7027,  102]])>,
 <tf.Tensor: id=90800, shape=(32, 102), dtype=int64, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])>]

In [55]:
np.array(x1_in).shape

(2, 32, 102)

In [119]:
x2_in.shape

TensorShape([32, 1])

In [ ]:
tf.reshape()

In [85]:
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam


bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

for l in bert_model.layers:
    l.trainable = True

# x1_in = Input(shape=(None,))
# x2_in = Input(shape=(None,))

#[x1_in, x2_in]=([X1=indices, X2=segments], Y)
#x1_in=[X1,X2]
#x2_in=Y



x = bert_model([x1_in, x2_in])

print(x.shape)
##shape=(None, None, 768)=（batch_size,seq_length,hidden_size）
##QA那个就可以接在这个后面，预训练情况下，用bert的encoder端结构足够了，本身就很复杂


ValueError: Layer model_15 was called with an input that isn't a symbolic tensor. Received type: <class 'list'>. Full input: [[<tf.Tensor: id=90799, shape=(32, 102), dtype=int64, numpy=
array([[ 101,    1, 5885, ...,    0,    0,    0],
       [ 101,  147,  124, ...,    0,    0,    0],
       [ 101,  100,  100, ...,    0,    0,    0],
       ...,
       [ 101, 4696, 3221, ..., 1990, 2823,  102],
       [ 101, 6968, 6228, ...,    0,    0,    0],
       [ 101,  966, 2533, ...,  741, 7027,  102]])>, <tf.Tensor: id=90800, shape=(32, 102), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>], <tf.Tensor: id=90801, shape=(32, 1), dtype=int64, numpy=
array([[0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       

In [90]:
type(bert_model)

keras.engine.training.Model

In [70]:
help(bert_model)

Help on Model in module keras.engine.training object:

class Model(keras.engine.network.Network)
 |  The `Model` class adds training & evaluation routines to a `Network`.
 |  
 |  Method resolution order:
 |      Model
 |      keras.engine.network.Network
 |      keras.engine.base_layer.Layer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  compile(self, optimizer, loss=None, metrics=None, loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None, **kwargs)
 |      Configures the model for training.
 |      
 |      # Arguments
 |          optimizer: String (name of optimizer) or optimizer instance.
 |              See [optimizers](/optimizers).
 |          loss: String (name of objective function) or objective function or
 |              `Loss` instance. See [losses](/losses).
 |              If the model has multiple outputs, you can use a different loss
 |              on each output by passing a dictionary or a list of losses.
 |         

In [104]:
# help(bert_model)

In [13]:
x

<tf.Tensor 'model_2/Encoder-12-FeedForward-Norm/add_1:0' shape=(None, None, 768) dtype=float32>

In [103]:
# ##留个问题以后想：就是经过padding之后，需要不需要输入mask矩阵，如果需要该如何输入
# input_ids = tf.constant([[31, 51, 99], [15, 5, 0]])
# input_mask = tf.constant([[1, 1, 1], [1, 1, 0]])
# token_type_ids = tf.Variable([[0, 0, 1], [0, 2, 0]])


# x1_input=[input_ids,token_type_ids]

# print(x1_input)
# # x2_input=None

# bert_model([x1_input,x2_input])

# # import tensorflow as tf
# # x_indices=[[1,2,3,0,0],[10,8,0,0,0]]
# # x_segments=[[0,0,0,0,0],[0,0,0,0,0]]
# # x_masks=[[1,1,1,0,0],[[1,1,0,0,0]]]
# # yy=[[0],[1]]
# # input_data=tf.constant([x_indices,x_segments])
# # print(input_data.shape)
# # x_temp = bert_model(input_data)

In [77]:
[xx,yy]=[[1,2,3],[3,4,5]],[9999]
print(xx,'\n',yy)

[[1, 2, 3], [3, 4, 5]] 
 [9999]


In [14]:
##这里的x是bert的output即self.sequence_output = self.all_encoder_layers[-1]

x = Lambda(lambda x: x[:, 0])(x)## 取出[CLS]对应的向量用来做分类
x#shape=(None, 768) =（batch_size,hidden_size)seq取第一个
####这个可以用于分类代表一句话的语义
#但是不能用于QA因为QA需要seq中每个token的语义，帮助判断是否为答案

<tf.Tensor 'lambda_1/strided_slice:0' shape=(None, 768) dtype=float32>

In [49]:
dd=np.array([
    [[1,2,3,4],[5,6,7,8],[9,10,11,12]],
     [[4,3,2,1],[8,7,6,5],[12,11,10,9]],
])
#(batch_size,seq_length,hidden_size)=(2,3,4)
print(dd.shape)
print(dd,'\n')
print(dd[:,0],'\n')##确实取了seq_0对应的向量#shape=(2,4)##理解成(2,3*4);前面的2都用到，3*4的二维矩阵只取第0行（所有列）
print(dd[:, 0:1, :],'\n')#shape=(2,1,4)
print(np.squeeze(dd[:, 0:1, :], axis=1))#squeeze之后变成和前面一样的了

(2, 3, 4)
[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]]

 [[ 4  3  2  1]
  [ 8  7  6  5]
  [12 11 10  9]]] 

[[1 2 3 4]
 [4 3 2 1]] 

[[[1 2 3 4]]

 [[4 3 2 1]]] 

[[1 2 3 4]
 [4 3 2 1]]


In [51]:
dd[:,0]

array([[1, 2, 3, 4],
       [4, 3, 2, 1]])

In [15]:
p = Dense(1, activation='sigmoid')(x)

model = Model([x1_in, x2_in], p)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-5), # 用足够小的学习率
    metrics=['accuracy']
)
model.summary()




Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]              

In [30]:
101677056/768

132392.0

In [46]:
132392./32

4137.25

In [52]:
train_D = data_generator(train_data)
valid_D = data_generator(valid_data)

In [81]:
for index, each in enumerate(valid_D):
    if index>0:
        break
    print(each)
#     print(each[0])
    print(np.array(each[1]).shape)
#     each[0],'\n',each[1]
#     print(each[0],'\n',each[1],'\n',each[2])
    

([array([[ 101,  704, 1744, ...,    0,    0,    0],
       [ 101, 2791, 7313, ...,    0,    0,    0],
       [ 101, 5632,  794, ...,    0,    0,    0],
       ...,
       [ 101, 1146,  775, ...,    0,    0,    0],
       [ 101, 3683,  704, ...,    0,    0,    0],
       [ 101,  122,  119, ..., 3683, 1071,  102]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])], array([[0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1]]))
(32, 1)


In [16]:


model.fit_generator(
    train_D.__iter__(),
    steps_per_epoch=len(train_D),
    epochs=5,
    validation_data=valid_D.__iter__(),
    validation_steps=len(valid_D)
)


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 11/594 [..............................] - ETA: 1:03:26 - loss: 0.6002 - accuracy: 0.6989

KeyboardInterrupt: 

In [91]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]              

In [94]:
ddd=model.get_layer('lambda_1').output_shape

In [95]:
ddd

(None, 768)

In [32]:
len(ddd)

0

In [35]:
ddd[0].shape

IndexError: list index out of range

In [27]:
ddd[1].shape

(2, 768)

In [29]:
ddd[3].shape

(768,)

In [18]:
model.predict_generator(valid_D.__iter__(),steps=10
    )

array([[0.7399683 ],
       [0.8641331 ],
       [0.32531452],
       [0.37061012],
       [0.62136763],
       [0.6888844 ],
       [0.7299098 ],
       [0.868304  ],
       [0.14410311],
       [0.8372526 ],
       [0.34206262],
       [0.2529854 ],
       [0.92776513],
       [0.3182729 ],
       [0.30037963],
       [0.42912117],
       [0.89562416],
       [0.59108394],
       [0.73442155],
       [0.16221812],
       [0.93979126],
       [0.28815824],
       [0.8344465 ],
       [0.5298022 ],
       [0.4722598 ],
       [0.25225955],
       [0.86818075],
       [0.7982975 ],
       [0.90290093],
       [0.46055615],
       [0.3619317 ],
       [0.31031817],
       [0.8076936 ],
       [0.8234111 ],
       [0.51733804],
       [0.8359425 ],
       [0.3521155 ],
       [0.17739475],
       [0.6987046 ],
       [0.33018547],
       [0.91324604],
       [0.86131847],
       [0.72156215],
       [0.9291462 ],
       [0.34599152],
       [0.12644461],
       [0.71060646],
       [0.861